In [23]:
import pandas as pd
import warnings
from scipy.stats import fisher_exact
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [24]:
# statstics on E. coli strains
total_stat = pd.read_csv('total_stats.csv', sep=',')
total_stat.head()

,AssemblyID,AssemblyName,Organism,Type,GenBankID,RefSeqID,length(bp),CDSs,MobElemCount,ISTransposaseCount(GB),ISTransposaseCount(RF),ISTransposaseCount(ISsaga),Cluster,Info
0,GCA_002310615.1,ASM231061v1,Escherichia coli 1190,Chromosome,CP023386.1,NZ_CP023386.1,4900891,4882,0.0,35,36,46/57,1 (D),NaN
1,GCA_002310615.1,ASM231061v1,Escherichia coli 1190,Plasmid,CP023387.1,NZ_CP023387.1,86147,112,0.0,0,0,NaN,1 (D),NaN
2,GCA_002310595.1,ASM231059v1,Escherichia coli 144,Chromosome,CP023364.1,NZ_CP023364.1,5134443,5049,0.0,61,61,82/96,1 (D),NaN
3,GCA_002310595.1,ASM231059v1,Escherichia coli 144,Plasmid,CP023365.1,NZ_CP023365.1,92421,127,0.0,1,1,NaN,1 (D),NaN
4,GCA_002310595.1,ASM231059v1,Escherichia coli 144,Plasmid,CP023363.1,NZ_CP023363.1,134388,180,0.0,12,12,NaN,1 (D),NaN


In [27]:
total_stat.Info.value_counts()

Info
STEC           119
shigellosis    110
ExPEC           86
ETEC            34
APEC            18
EPEC            10
AIEC             5
EAEC             3
EIEC             3
Name: count, dtype: int64

In [25]:
# metadata on yih loci composition
yih_homologs_loci = pd.read_csv('yih_homologs_loci.csv', 
                                sep='\t')


In [28]:
yih_homologs_loci['variant'] = yih_homologs_loci.set_queryGenes_locus.apply(lambda x: 
                                             'long' if 'yihQ' in x or 'yihS' in x else 'short')

In [29]:
yih_homologs_loci['Info']=yih_homologs_loci.genome_id.\
map(dict(zip(total_stat.AssemblyID, total_stat.Info)))

In [30]:
# prepare data
yih_homologs_loci_drop = yih_homologs_loci.\
drop_duplicates('set_locusTag_locus') #.Info.value_counts()

yih_homologs_loci_drop['patho_status'] = yih_homologs_loci_drop.Info
yih_homologs_loci_drop['patho_status'] = yih_homologs_loci_drop['patho_status'].\
fillna( 'non-pathpgenic')
yih_homologs_loci_drop['patho_status'] = yih_homologs_loci_drop['patho_status'].\
apply(lambda x: 'non-pathpgenic' if x == 'non-pathpgenic' else 'pathpgenic')

In [31]:
# construct contingency table
contingency_table = pd.crosstab(
    yih_homologs_loci_drop["patho_status"], 
    yih_homologs_loci_drop["variant"]
)
print(contingency_table)

# construct fisher_exact test
odds_ratio, p_value = fisher_exact(contingency_table)
print(f"Odds Ratio = {odds_ratio:.2f}, p-value = {p_value:.9f}")


variant         long  short
patho_status               
non-pathpgenic   202     78
pathpgenic        63     65
Odds Ratio = 2.67, p-value = 0.000011212
